<a href="https://colab.research.google.com/github/Carl-Zzdr/MLstudy/blob/main/HW%EF%BC%8D241027%EF%BC%8D2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 导入必要的库
import pickle
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization, Reshape, ReLU
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import tensorflow as tf

# 读取数据集
faults = ['12DriveEndFault0.007-Ball_211',
          '12DriveEndFault0.007-InnerRace_214',
          '12DriveEndFault0.007-OuterRace6_217',
          '12DriveEndFault0.014-InnerRace_225',
          '12DriveEndFault0.021-InnerRace_226']
dataset = []
path = '/content/drive/MyDrive/GZdata/'  # 数据路径

for fault in faults:
    with open(path + fault + '.pkl', 'rb') as f:
        data = pickle.load(f)
    dataset.extend(data)

# 数据处理
dataset_x = np.array(dataset)
label_y = np.array([0]*600 + [1]*600 + [2]*600 + [3]*600 + [4]*600)

# 划分训练和测试集
X_train, X_test, y_train, y_test = train_test_split(dataset_x, label_y, test_size=0.3, random_state=42)

# 定义卷积神经网络（仅用于特征提取）
def create_cnn_feature_extractor(input_shape=(1024,)):
    in_image = Input(shape=input_shape)
    g = Reshape((1024, 1))(in_image)
    g = Conv1D(16, 5, strides=2, padding='same')(g)
    g = BatchNormalization()(g)
    g = ReLU(0.3)(g)
    g = Conv1D(8, 5, strides=2, padding='same')(g)
    g = BatchNormalization()(g)
    g = ReLU(0.3)(g)
    g = Conv1D(4, 5, strides=2, padding='same')(g)
    g = BatchNormalization()(g)
    g = ReLU(0.3)(g)
    g = Conv1D(2, 5, strides=2, padding='same')(g)
    g = BatchNormalization()(g)
    g = ReLU(0.3)(g)
    g = Flatten()(g)
    feature_output = Dense(256, activation='sigmoid')(g)

    model = Model(inputs=in_image, outputs=feature_output)
    return model

# 构建特征提取模型
cnn_feature_extractor = create_cnn_feature_extractor()
cnn_feature_extractor.summary()

# 提取训练和测试集的特征
X_train_features = cnn_feature_extractor.predict(X_train)
X_test_features = cnn_feature_extractor.predict(X_test)

# 逻辑回归分类器
log_reg = LogisticRegression(max_iter=100)  # 运行100步
log_reg.fit(X_train_features, y_train)

# 测试集上的预测
y_pred = log_reg.predict(X_test_features)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy with Logistic Regression: {accuracy:.4f}")


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 1024, 1)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 512, 16)             │              96 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 512, 16)             │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu (ReLU)                         │ (None, 512, 16)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 256, 8)              │             648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 256, 8)              │              32 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_1 (ReLU)                       │ (None, 256, 8)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 128, 4)              │             164 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 128, 4)              │              16 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_2 (ReLU)                       │ (None, 128, 4)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 64, 2)               │              42 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 64, 2)               │               8 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_3 (ReLU)                       │ (None, 64, 2)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │          33,024 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,094 (133.18 KB)

 Trainable params: 34,034 (132.95 KB)

 Non-trainable params: 60 (240.00 B)

66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
Test Accuracy with Logistic Regression: 0.2156
